# **Project 2**
### Josh Winnes and Jon Contreras

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

import geopandas as gpd
import shapely
import skmob
from skmob.measures.individual import jump_lengths
from skmob.tessellation import tilers
from skmob.preprocessing import detection, clustering
from skmob.preprocessing import filtering

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)
import warnings
warnings.simplefilter("ignore")

import os

try:   
    os.chdir('/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/Wheaton College/fall 2024/topics in data science/data')
    kmz_file_josh = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/Wheaton College/fall 2024/topics in data science/data/March 7, 2024 - Whistler Blackcomb(Josh).kmz'
    #kmz_file_jon = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/Wheaton College/fall 2024/topics in data science/data/March 7, 2024 - Whistler Blackcomb(Jon).kmz'
    
    import sys
    script_directory = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/Wheaton College/fall 2024/topics in data science/Math485/Project2'
    sys.path.append(script_directory)
    from gpx_kmz_util import *
    
    print('Working from Josh\'s laptop')
except FileNotFoundError:
    os.chdir('/Users/jonathancontreras/Desktop/College Folder/Senior Year/Math485/Unit 1 Tabular/archive')
    kmz_file_josh = '/Users/jonathancontreras/Desktop/College Folder/Senior Year/Math485/Unit 1 Tabular/archive/data/March 7, 2024 - Whistler Blackcomb(Josh).kmz'
    kmz_file_jon = '/Users/jonathancontreras/Desktop/College Folder/Senior Year/Math485/Unit 1 Tabular/archive/data/March 7, 2024 - Whistler Blackcomb(Jon).kmz'
    
    import sys
    script_directory = 'path_to_your_directory_with_gpx_kmz_util'
    sys.path.append(script_directory)
    from gpx_kmz_util import *
    
    print('Working from Jonathan\'s laptop')

Working from Josh's laptop


In [2]:
df_jon_gpx = gpx_to_df('March 7, 2024 - Whistler Blackcomb(Jon).gpx')
df_josh_gpx = gpx_to_df('March 7, 2024 - Whistler Blackcomb(Josh).gpx')

kml_data_josh = extract_kml_from_kmz(kmz_file_josh)
df_josh_kmz = parse_kml_to_dataframe(kml_data_josh)

#kml_data_jon = extract_kml_from_kmz(kml_file_jon)
#df_jon_kml = parse_kml_to_dataframe(kml_data_jon)

In [3]:
df_jon_gpx.head()

,Latitude,Longitude,Elevation,Time
0,50.111385,-122.951516,699.522521,2024-03-07 09:52:51-08:00
1,50.109740,-122.953332,709.858664,2024-03-07 09:52:52-08:00
2,50.109765,-122.953274,741.954162,2024-03-07 09:52:53-08:00
3,50.109679,-122.953279,746.486242,2024-03-07 09:52:55-08:00
4,50.109586,-122.953275,748.005084,2024-03-07 09:52:58-08:00


In [4]:
df_josh_gpx.head()

,Latitude,Longitude,Elevation,Time
0,50.113260,-122.953950,683.549635,2024-03-07 09:46:36-08:00
1,50.113391,-122.953915,686.117031,2024-03-07 09:46:36-08:00
2,50.113319,-122.953772,688.737785,2024-03-07 09:48:27-08:00
3,50.113267,-122.953655,689.777724,2024-03-07 09:48:32-08:00
4,50.113181,-122.953594,692.476930,2024-03-07 09:49:04-08:00


In [5]:
df_josh_kmz

,name,description,longitude,latitude
0,Lift 1,None,-122.953366,50.113067
1,Run 1,None,-122.951056,50.078425
2,Lift 2,None,-122.963743,50.085441
3,Run 2,None,-122.947029,50.067096
4,Lift 3,None,-122.952236,50.066824
5,Run 3,None,-122.957573,50.059251
6,Lift 4,None,-122.964118,50.085837
7,Run 4,None,-122.947552,50.066370
8,Lift 5,None,-122.952248,50.066714
9,Run 5,None,-122.957416,50.059398


### ***Compiling Data into 1 DataFrame***